In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
train_set = pd.read_csv('train.csv')
train_set.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y7,x8,y8,x9,y9,x10,y10,speed,throttle,steering
0,116.689494,434.717131,302.499051,1276.520563,488.308608,2118.323995,674.118164,2960.127428,859.927721,3801.930860,...,4457.924736,2915.153699,5113.918612,3942.766688,5769.912487,4314.385802,7453.519352,62.436092,1.0,1.0
1,-509.950872,404.630562,-183.895471,1096.986188,468.215331,2481.697440,1120.326132,3866.408691,1772.436934,5251.119943,...,6982.009008,2750.603136,7328.186821,3259.808649,7511.336933,3769.014162,7694.487046,106.698936,1.0,1.0
2,-557.612922,420.921138,627.905866,1474.735991,1813.424653,2528.550845,2998.943440,3582.365698,4184.462228,4636.180552,...,6743.810258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55.832943,0.0,1.0
3,-174.353965,382.044942,-50.503595,1553.527709,73.346775,2725.010476,-450.469423,3372.677045,-974.285621,4020.343614,...,6611.009889,-4117.182812,7906.343026,0.000000,0.000000,0.000000,0.000000,82.067360,1.0,0.0
4,-445.653146,73.830818,-269.683183,874.845971,82.256744,2476.876276,434.196672,4078.906581,2036.226977,3726.966654,...,3023.086800,6842.317893,2671.146873,8444.348199,2319.206946,0.000000,0.000000,108.571968,0.0,1.0


In [3]:
train_labels = train_set.iloc[:, -2:]
train_labels.head()

,throttle,steering
0,1.0,1.0
1,1.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0


In [4]:
def combineClasses(throttle, steering):
    if throttle == -1 and steering == -1:
        return 0
    if throttle == -1 and steering == 0:
        return 1
    if throttle == -1 and steering == 1:
        return 2
    if throttle == 0 and steering == -1:
        return 3
    if throttle == 0 and steering == 0:
        return 4
    if throttle == 0 and steering == 1:
        return 5
    if throttle == 1 and steering == -1:
        return 6
    if throttle == 1 and steering == 0:
        return 7
    if throttle == 1 and steering == 1:
        return 8
    

In [5]:
train_combined_labels = train_labels.apply(lambda row: combineClasses(row['throttle'], row['steering']), axis=1)
train_combined_labels.head()

0    8
1    8
2    5
3    7
4    5
dtype: int64

In [6]:
train_set_features = train_set.iloc[:,:-2]
train_set_features.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y6,x7,y7,x8,y8,x9,y9,x10,y10,speed
0,116.689494,434.717131,302.499051,1276.520563,488.308608,2118.323995,674.118164,2960.127428,859.927721,3801.930860,...,4129.927798,1887.540710,4457.924736,2915.153699,5113.918612,3942.766688,5769.912487,4314.385802,7453.519352,62.436092
1,-509.950872,404.630562,-183.895471,1096.986188,468.215331,2481.697440,1120.326132,3866.408691,1772.436934,5251.119943,...,6635.831195,2587.575435,6982.009008,2750.603136,7328.186821,3259.808649,7511.336933,3769.014162,7694.487046,106.698936
2,-557.612922,420.921138,627.905866,1474.735991,1813.424653,2528.550845,2998.943440,3582.365698,4184.462228,4636.180552,...,5689.995405,6555.499803,6743.810258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55.832943
3,-174.353965,382.044942,-50.503595,1553.527709,73.346775,2725.010476,-450.469423,3372.677045,-974.285621,4020.343614,...,5315.676751,-3069.550415,6611.009889,-4117.182812,7906.343026,0.000000,0.000000,0.000000,0.000000,82.067360
4,-445.653146,73.830818,-269.683183,874.845971,82.256744,2476.876276,434.196672,4078.906581,2036.226977,3726.966654,...,3375.026727,5240.287588,3023.086800,6842.317893,2671.146873,8444.348199,2319.206946,0.000000,0.000000,108.571968


In [7]:
normalizer = tf.keras.layers.Normalization(axis=1)
normalizer.adapt(train_set_features)

In [8]:
def get_basic_model2():
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(150, activation='relu'),
        tf.keras.layers.Dense(9)
    ])
    
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [9]:
model = get_basic_model2()
model.fit(train_set_features, train_combined_labels, epochs=500, batch_size=10000)

Epoch 1/500
41/41 [==============================] - 4s 94ms/step - loss: 1.4804 - accuracy: 0.4526
Epoch 2/500
41/41 [==============================] - 4s 93ms/step - loss: 1.0835 - accuracy: 0.5622
Epoch 3/500
41/41 [==============================] - 4s 99ms/step - loss: 1.0130 - accuracy: 0.5884
Epoch 4/500
41/41 [==============================] - 5s 110ms/step - loss: 0.9740 - accuracy: 0.6045
Epoch 5/500
41/41 [==============================] - 5s 114ms/step - loss: 0.9466 - accuracy: 0.6150
Epoch 6/500
41/41 [==============================] - 4s 92ms/step - loss: 0.9247 - accuracy: 0.6238
Epoch 7/500
41/41 [==============================] - 4s 91ms/step - loss: 0.9092 - accuracy: 0.6292
Epoch 8/500
41/41 [==============================] - 4s 92ms/step - loss: 0.8918 - accuracy: 0.6354
Epoch 9/500
41/41 [==============================] - 4s 100ms/step - loss: 0.8829 - accuracy: 0.6385
Epoch 10/500
41/41 [==============================] - 3s 76ms/step - loss: 0.8675 - accuracy: 0.6

In [10]:
test_set = pd.read_csv('test.csv')
test_set.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y7,x8,y8,x9,y9,x10,y10,speed,throttle,steering
0,-336.633569,1812.060034,-277.878680,2422.565944,-219.123790,3033.071853,-770.874810,3702.332652,-1322.625831,4371.593451,...,5710.115049,-2977.878891,6379.375848,-3529.629911,7048.636647,-4140.135820,7107.391537,115.162102,0.0,-1.0
1,-226.376966,912.846314,-185.093845,2889.968992,-143.810724,4867.091671,-102.527602,6844.214350,-61.244481,8821.337028,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.224815,1.0,0.0
2,9662.330866,1699.190841,9057.538183,431.489922,-363.045379,257.143259,241.747303,1524.844178,846.539986,2792.545098,...,5327.946936,1724.671232,6264.193737,1393.217114,7200.440538,730.308878,9072.934139,76.790726,1.0,1.0
3,217.142292,652.620238,68.465416,1952.585739,644.109728,2676.906928,1219.754040,3401.228116,1795.398353,4125.549305,...,6149.835995,2073.688912,7449.801497,1925.012036,8749.766998,0.000000,0.000000,92.042717,0.0,1.0
4,48.802369,9.829263,48.769464,616.159568,48.736559,1222.489872,-557.626651,1828.787271,-1163.989860,2435.084671,...,3647.679469,-2983.079490,4253.976868,-3589.442699,4860.274267,-4195.773004,4860.241362,0.004658,1.0,0.0


In [11]:
test_features = test_set.iloc[:,:-2]
test_features.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y6,x7,y7,x8,y8,x9,y9,x10,y10,speed
0,-336.633569,1812.060034,-277.878680,2422.565944,-219.123790,3033.071853,-770.874810,3702.332652,-1322.625831,4371.593451,...,5040.854250,-2426.127871,5710.115049,-2977.878891,6379.375848,-3529.629911,7048.636647,-4140.135820,7107.391537,115.162102
1,-226.376966,912.846314,-185.093845,2889.968992,-143.810724,4867.091671,-102.527602,6844.214350,-61.244481,8821.337028,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.224815
2,9662.330866,1699.190841,9057.538183,431.489922,-363.045379,257.143259,241.747303,1524.844178,846.539986,2792.545098,...,4060.246017,2056.125351,5327.946936,1724.671232,6264.193737,1393.217114,7200.440538,730.308878,9072.934139,76.790726
3,217.142292,652.620238,68.465416,1952.585739,644.109728,2676.906928,1219.754040,3401.228116,1795.398353,4125.549305,...,4849.870494,2222.365789,6149.835995,2073.688912,7449.801497,1925.012036,8749.766998,0.000000,0.000000,92.042717
4,48.802369,9.829263,48.769464,616.159568,48.736559,1222.489872,-557.626651,1828.787271,-1163.989860,2435.084671,...,3041.382070,-2376.716280,3647.679469,-2983.079490,4253.976868,-3589.442699,4860.274267,-4195.773004,4860.241362,0.004658


In [12]:
test_labels = test_set.iloc[:,-2:]
test_labels.head()

,throttle,steering
0,0.0,-1.0
1,1.0,0.0
2,1.0,1.0
3,0.0,1.0
4,1.0,0.0


In [13]:
test_combined_labels = test_labels.apply(lambda row: combineClasses(row['throttle'], row['steering']), axis=1)
test_combined_labels.head()

0    3
1    7
2    8
3    5
4    7
dtype: int64

In [14]:
results = model.evaluate(test_features, test_combined_labels)
print('test loss, test:', results)

2742/2742 [==============================] - 4s 1ms/step - loss: 0.2374 - accuracy: 0.9179
test loss, test: [0.23742614686489105, 0.9179404973983765]


In [15]:
model.save_weights(filepath='./NNdata/weights')

In [16]:
test_model = get_basic_model2()
test_model.load_weights('./NNdata/weights')

In [17]:
results = test_model.evaluate(test_features, test_combined_labels)
print('test loss, test:', results)

2742/2742 [==============================] - 4s 1ms/step - loss: 0.2374 - accuracy: 0.9179
test loss, test: [0.23742614686489105, 0.9179404973983765]


In [18]:
#needs to be manually copied to ml_controller 'LEARNEND MEANS' and 'LEARNED VARIANCES'
normalizer.variables

[<tf.Variable 'mean:0' shape=(21,) dtype=float32, numpy=
 array([ 2.9114618e+01,  9.0919641e+02,  2.2618099e+01,  2.0540454e+03,
         2.2350567e+01,  3.1214478e+03,  3.0489363e+01,  4.1120122e+03,
         3.6451164e+01,  4.6826846e+03,  1.5953860e+01,  4.9184155e+03,
        -1.8753021e+01,  4.8326768e+03, -3.4846458e+01,  4.4459541e+03,
        -1.4296617e+01,  3.8252571e+03, -4.7434478e+00,  3.1695378e+03,
         9.8633614e+01], dtype=float32)>,
 <tf.Variable 'variance:0' shape=(21,) dtype=float32, numpy=
 array([1.5271010e+06, 1.6488716e+06, 1.0609779e+06, 1.6856356e+06,
        1.1636901e+06, 2.3239340e+06, 1.9838620e+06, 3.3765532e+06,
        3.5502898e+06, 4.3483820e+06, 5.3716085e+06, 5.7465245e+06,
        6.7475600e+06, 7.9761805e+06, 7.1915225e+06, 1.0374997e+07,
        7.0851285e+06, 1.1882636e+07, 7.0572995e+06, 1.2012724e+07,
        6.9620044e+02], dtype=float32)>,
 <tf.Variable 'count:0' shape=() dtype=int64, numpy=409399>]